# Prototype - *get_patch_location_array()* function
    * Common to all 4 patch extraction use cases.
    * Decouples selection image mask & (mask size) from individual use case flows.
    * Insertion point for new or modified location-selection methods.
    * Allow inspection / select-method parameter modification before making output.
    * Allows paralellization of patch location (think slower more specific select methods). <br>
****
## Assemble and create unit test data:

In [1]:
import time
import os
import sys
import openslide

sys.path.insert(0, '../src/python')
from digipath_toolkit import *

data_dir = '../../DigiPath_MLTK_data/Aperio'
file_type_list=['.svs', '.tif', '.tiff']
fs_od = get_file_size_ordered_dict(data_dir, file_type_list)
list_number = 0
for k, v in fs_od.items():
    print('%3i %30s: %i'%(list_number, k,v))
    list_number += 1

  0         CMU-1-Small-Region.svs: 1938955
  1               JP2K-33003-1.svs: 63847265
  2           CMU-1-JP2K-33005.svs: 132565343
  3                      CMU-1.svs: 177552579
  4                      CMU-3.svs: 253815723
  5               JP2K-33003-2.svs: 289250433
  6                      CMU-2.svs: 390750635


In [3]:
#                         test data file for threshold_otsu
run_parameters = {}

data_dir = '../data/images/' 
image_file_name = 'CMU-1-Small-Region.svs'
run_parameters['wsi_filename'] = os.path.join(data_dir, image_file_name)
print('Image File:\n', run_parameters['wsi_filename'])

run_parameters['thumbnail_divisor'] = 10
run_parameters['patch_select_method'] = 'threshold_otsu' # 'threshold_rgb2lab'
run_parameters['patch_height'] = 224
run_parameters['patch_width'] = 224

"""
                output & time test:                CALL - - get_patch_location_array()
"""
t0 = time.time()
patch_location_array = get_patch_location_array(run_parameters)
tt = time.time() - t0

print('%i images found\t%0.3f s\n'%(len(patch_location_array), tt))

print('type(patch_location_array) =',type(patch_location_array), '\t', len(patch_location_array),'\n')
for d in patch_location_array:
    print(type(d), '\t', d)

Image File:
 ../data/images/CMU-1-Small-Region.svs
74 images found	0.424 s

type(patch_location_array) = <class 'list'> 	 74 

<class 'tuple'> 	 (28, 774)
<class 'tuple'> 	 (28, 998)
<class 'tuple'> 	 (28, 1222)
<class 'tuple'> 	 (252, 774)
<class 'tuple'> 	 (252, 998)
<class 'tuple'> 	 (252, 1222)
<class 'tuple'> 	 (252, 1446)
<class 'tuple'> 	 (476, 774)
<class 'tuple'> 	 (476, 998)
<class 'tuple'> 	 (476, 1222)
<class 'tuple'> 	 (476, 1446)
<class 'tuple'> 	 (476, 1894)
<class 'tuple'> 	 (700, 774)
<class 'tuple'> 	 (700, 998)
<class 'tuple'> 	 (700, 1222)
<class 'tuple'> 	 (700, 1446)
<class 'tuple'> 	 (924, 102)
<class 'tuple'> 	 (924, 326)
<class 'tuple'> 	 (924, 550)
<class 'tuple'> 	 (924, 774)
<class 'tuple'> 	 (924, 998)
<class 'tuple'> 	 (924, 1222)
<class 'tuple'> 	 (924, 1446)
<class 'tuple'> 	 (924, 1670)
<class 'tuple'> 	 (924, 1894)
<class 'tuple'> 	 (1148, 774)
<class 'tuple'> 	 (1148, 998)
<class 'tuple'> 	 (1148, 1222)
<class 'tuple'> 	 (1148, 1446)
<class 'tuple'> 	